# Osztályozási feladat

Olvasd el az [előadás olvasóleckét](http://inf.u-szeged.hu/~rfarkas/ML20/osztalyozas.html)!


## Adatelőkészítés
Ebben a leckében az [UCI Credit Approval](https://archive.ics.uci.edu/ml/datasets/Credit+Approval) adatbázist fogjuk használni és azt akarjuk osztályozni, hogy egy hitelkártya igénylés teljesíthető (`class=+`) vagy elutasítandó (`class=-`).

In [ ]:
import pandas as pd
df = pd.read_csv('https://datahub.io/machine-learning/credit-approval/r/credit-approval.csv') 
df

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,class
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


In [ ]:
classlabel = df["class"] # osztálycímke (célváltozó), amit előre akarunk jelezni
features = df.iloc[:,:-1] # az utolsó oszlop kivételével minden oszlop jellemző (ezek alapján történik az előrejelzés)

In [ ]:
classlabel.value_counts()

-    383
+    307
Name: class, dtype: int64

In [ ]:
classlabel.value_counts()['-'] / len(classlabel)

0.5550724637681159

In [ ]:
from sklearn import tree
dt = tree.DecisionTreeClassifier() # döntési fa osztályozó

In [ ]:
# egy osztályozó tanítása a fit() függvénnyel a jellemzővektorok és osztálycímkék (célváltozó) alapján
dt.fit(features, classlabel)

ValueError: ignored

## Diszkrét jellemzők kódolása

Ez így sajnos nem jó, mert a kategórikus jellemzőket nem tudja defaultból kezelni az sklearn :(

Csak folytonos változókat képes... Át kell kódolnunk, használjuk a one hot encodingot! 

In [ ]:
from sklearn import preprocessing
# ha csak a kategórikus változókat one hot encodoljuk:
ohe = preprocessing.OneHotEncoder() #one hot encoding
cat_features = ohe.fit_transform(features.select_dtypes(include=['object'])) # kategórikus jellemzők ritka mátrixként
import scipy.sparse
num_features = scipy.sparse.csr_matrix(features.select_dtypes(exclude=['object'])) # numerikus jellemzők ritka mátrixként
ohe_features = scipy.sparse.hstack((cat_features, num_features)) # ritka mátrixok horizontális konkatenálása 
ohe_features

In [ ]:
cat_features

In [ ]:
features = features.dropna()

In [ ]:
dt = tree.DecisionTreeClassifier()
dt.fit(ohe_features, classlabel)

## Döntési fa osztályozó
[Döntési fa osztályozót](http://inf.u-szeged.hu/~rfarkas/ML20/dontesi_fa.html) fogunk használni, ami jó választás kevés számú diszkrét jellemző esetén.

In [ ]:
# Vizualizáljuk a megtanult döntési fát!
# Ehhez le kell töltenünk egy extra csomagot a colabba:
!apt-get -qq install -y graphviz && pip install -q pydot #Thank you https://medium.com/deep-learning-turkey/google-colab-free-gpu-tutorial-e113627b9f5d
!pip install graphviz

In [ ]:
import pydot
import graphviz

In [ ]:
# dt megjelenítése
graphviz.Source(tree.export_graphviz(dt, out_file=None))

In [ ]:
# döntési fa mélységének korlátozása változtatása:
dt = tree.DecisionTreeClassifier(min_samples_leaf=20, max_depth=2) # legfeljebb 3 mély fát építhet
dt.fit(ohe_features, classlabel)
graphviz.Source(tree.export_graphviz(dt, out_file=None))

## Kiértékelés

In [ ]:
prediction = dt.predict(ohe_features)
prediction

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(prediction, classlabel)

Mindig hasonlítsuk a gépi tanulás eredményét [baseline](http://inf.u-szeged.hu/~rfarkas/ML20/baseline.html) döntésekhez!


In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent") # tanító adatbázis leggyakoribb osztálya lesz mindig a predikció
dummy_clf.fit(features, classlabel) # ugyanazon a tanító adatbázison "tanítjuk"
baseline_prediction = dummy_clf.predict(features) # predikció a kiértékelő adatbázison
accuracy_score(baseline_prediction, classlabel)

## Predikció korábban nem látott példákra

[Osztályozók kiértékelését](http://inf.u-szeged.hu/~rfarkas/ML20/osztalyozas.html) a független kiértékelő adatbázison kell végezni.

In [ ]:
from sklearn.model_selection import train_test_split
features_train, features_test, classlabel_train, classlabel_test = train_test_split(ohe_features, classlabel, test_size=0.25, random_state=42)

In [ ]:
# döntési fa tanítása
d = tree.DecisionTreeClassifier(max_leaf_nodes=10)
d.fit(features_train, classlabel_train)

In [ ]:
prediction = d.predict(features_test)
prediction

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(prediction, classlabel_test)

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent") # tanító adatbázis leggyakoribb osztálya lesz mindig a predikció
dummy_clf.fit(features_train, classlabel_train) # ugyanazon a tanító adatbázison "tanítjuk"
baseline_prediction = dummy_clf.predict(features_test) # predikció a kiértékelő adatbázison
accuracy_score(baseline_prediction, classlabel_test)

## Osztályonkénti metrika

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(classlabel_test, prediction))

## k-szoros keresztvalidáció

In [ ]:
from sklearn.model_selection import cross_val_score
dt = tree.DecisionTreeClassifier()
cross_val_score(dt, ohe_features, classlabel, cv=10)

In [ ]:
cross_val_score(dt, ohe_features, classlabel, cv=10).mean()

In [ ]:
from sklearn import metrics
cross_val_score(dt, ohe_features, classlabel, cv=10, scoring='f1_micro')

# Gyakorló fealdatok

1. Értékeld ki az 1, 2, 3 mélységűre korlátozott döntési fákat ezen az adatbázison!

2. Hajts végre egy osztályozási kísérletet a [survey adatbázison](https://stat.ethz.ch/R-manual/R-devel/library/MASS/html/survey.html), ahol azt akarjuk predikálni, hogy kézkulcsolásnál melyik kéz van felül (`Fold`), az összes többi oszlop, mint jellemző alapján!

